In [ ]:
# pip install numpy==1.26.4

In [1]:
import numpy as np
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.model_selection import GridSearchCV, cross_validate
import warnings
warnings.filterwarnings('ignore')



In [2]:
from surprise.model_selection import train_test_split
from surprise import accuracy

In [3]:
movies = pd.read_csv("data/movies.csv")
links = pd.read_csv("data/links.csv")
ratings = pd.read_csv("data/ratings.csv")
tags = pd.read_csv("data/tags.csv")

In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [22]:
train_df = ratings.drop(columns='timestamp')
train_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [19]:
# pip install tqdm


In [24]:


reader = Reader()

data = Dataset.load_from_df(train_df, reader)


#Instantiate an SVD model from the surprice package
svd = SVD(n_epochs=45,n_factors=1000,init_std_dev=0.001,random_state=25,verbose=True)

#Instanciate and build the dataset with data object from surprise package
trainset = data.build_full_trainset()

#Train the SVD model
svd.fit(trainset)



Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44


In [7]:

# Make predictions on the testset
predictions = algo.test(testset)

# Compute and print the RMSE (Root Mean Squared Error)
rmse = accuracy.rmse(predictions)

RMSE: 0.8733


In [ ]:
import pickle as pkl
pkl.dump(links, open('links.pkl', 'wb'))
pkl.dump(titles, open('titles.pkl', 'wb'))
pkl.dump(svd, open('svdmodel.pkl', 'wb'))

# For my recommender system i will use KNN 

In [29]:
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from collections import defaultdict

In [30]:
reader = Reader()

data = Dataset.load_from_df(train_df, reader)

# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2)

# Use the KNNBasic algorithm for item-based collaborative filtering
sim_options = {
    'name': 'cosine',
    'user_based': False  # Compute similarities between items
}
algo = KNNBasic(sim_options=sim_options, verbose=True)

# Train the algorithm on the trainset
algo.fit(trainset)



Computing the cosine similarity matrix...
Done computing similarity matrix.


In [46]:
# Function to get top N similar movies to the selected movie
def get_top_n_similar_movies(algo, movie_id, n=10):
    # Retrieve inner id of the selected movie
    inner_id = algo.trainset.to_inner_iid(movie_id)
    # Retrieve the k most similar items to the selected item
    neighbors = algo.get_neighbors(inner_id, k=n)
    # Convert inner ids of the neighbors to raw ids
    neighbors = [algo.trainset.to_raw_iid(inner_id) for inner_id in neighbors]
    return neighbors

# Function to get movie names from movie ids
def get_movie_names(recoms, movie_df):
    movie_names = [movie_df.loc[movie_df['movieId'] == int(movie_id), 'title'].values[0] for movie_id in recoms]
    return movie_names

# Example of recommending top 10 movies similar to movie with movieId=47
selected_movie_id = 47
similar_recoms = get_top_n_similar_movies(algo, selected_movie_id, n=10)
similar_movies = get_movie_names(similar_recoms, movies)

print(f"Movies similar to movieId {selected_movie_id}: {similar_movies}")


Movies similar to movieId 47: ['Sherlock Holmes: Dressed to Kill (1946)', "Bells of St. Mary's, The (1945)", 'Summer Rental (1985)', 'Captain Fantastic (2016)', 'A Quiet Place (2018)', 'Company of Wolves, The (1984)', 'Love Affair (1994)', 'Magnum Force (1973)', 'American Hardcore (2006)', 'Dragon Ball: The Path to Power (Doragon bôru: Saikyô e no michi) (1996)']


In [34]:
recom_model = algo

In [41]:
movies['title'][1-1]

'Toy Story (1995)'

In [37]:
titles = movies.drop(columns='genres')
titles.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [47]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [28]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
